# Exemplo Árvore de classificação com PCA

Neste exemplo será utilizado os dados da pesquisa sobre *Nerdy Personality Attributes Scale*

## Carregar pacotes

In [ ]:
library(tidyverse)
library(magrittr)
library(psych)
library(plotly)
library(rpart)
library(caret)

## Carregar dados

In [ ]:
load( '/home/vm-data-science/dados/dados_nerd_nomissings.RData' )

In [ ]:
dados_nerd_train %>% 
    head()

In [ ]:
dados_nerd_test %>% 
    head()

## Criação da classificação: 1 - nerd, 0 - não nerd

In [ ]:
dados_nerd_train %<>% 
  mutate( nerdy_classification = ifelse( nerdy > 5, 1, 0) )
 
dados_nerd_test %<>% 
  mutate( nerdy_classification = ifelse( nerdy > 5, 1, 0) )

## Análise exploratória

In [ ]:
# count em todas as colunas
dados_nerd_train %>% 
  map( ~count(data.frame(x=.x), x) )

In [ ]:
# outliers
dados_nerd_train %>% 
  dplyr::select( contains('Q') ) %>% 
  gather( key = variaveis, value = notas ) %>% 
  plot_ly( x = ~variaveis,
           y = ~notas,
           type = 'box' )

## Combinação de técnicas supervisionadas com não supervisionadas

1 - Aplicamos o algoritmo não supervisionado

2 - Geramos os novos atributos no banco de dados

3 - Aplicamos o algoritmo supervisionado com os atributos obtidos pelo não supervisionado

### 1 - Aplicamos o PCA

- Verificamos a correlação

In [ ]:
correlacao <- dados_nerd_train %>% 
  dplyr::select( contains('Q') ) %>% 
  cor() %>% 
  round(., 2)

In [ ]:
plot_ly( z = correlacao,
         x = colnames(correlacao),
         y = colnames(correlacao), 
         colorscale = "Greys", 
         type = "heatmap" )

- Aplicamos o PCA ( nfactors = 26 porque temos 26 questões)

In [ ]:
pca_model <- principal(dados_nerd_train %>% 
                         dplyr::select( contains('Q') ), 
                       nfactors = 26, 
                       rotate = "none" )

- Verificamos o *screeplot*

In [ ]:
pca_model$values %>% 
  as_data_frame %>% 
  rename( autovalor = value ) %>% 
  mutate( dimensao = 1:26 ) %>% 
  plot_ly(x = ~dimensao,
          y = ~autovalor,
          type = 'scatter',
          mode = 'lines+markers',
          marker = list(size = 10, color = 'red') )

- Verificamos a variância explicada

In [ ]:
pca_model$Vaccounted %>% 
  as.data.frame() %>% 
  rownames_to_column( var = 'medidas' ) %>% 
  gather( key = PC, value = valores, -medidas ) %>% 
  spread( key = medidas, value = valores ) %>% 
  arrange( desc(`SS loadings`) )

- Pela observação do *screeplot* percebe-se que a partir do componente 7 não há grande variação. Porém, caso 7 componentes sejam escolhidos somente 58% da informação é explicada por estes dados.


- Diante disto, buscamos adicionar os componentes 8 a 11 para que 70% da informação seja explicada pelos componentes.

### 2 - Geramos os novos atributos no banco de dados

In [ ]:
# Guardamos os componentes num objeto
scores_pca <- pca_model$scores[, 1:11] %>% 
  as_data_frame()

In [ ]:
# Adicionamos os componentes no banco
dados_nerd_train_modelo <- dados_nerd_train %>% 
  bind_cols(., scores_pca )

In [ ]:
dados_nerd_train_modelo %>% 
    head()

### 3 - Aplicamos o modelo de árvore de decisão

- Alguns ajustes importantes no banco de dados no R

In [ ]:
dados_nerd_train_modelo %<>% 
  mutate( gender = as.factor(gender),
          education = as.factor(education),
          married = as.factor(married),
          ASD = as.factor(ASD),
          nerdy_classification = as.factor(nerdy_classification) )

- Dividir a amostra de treinamento em: treino/validação

In [ ]:
# treino
set.seed(543)
dados_nerd_train_modelo_train <- dados_nerd_train_modelo %>% 
  sample_frac(., 0.8)

In [ ]:
# validacao
dados_nerd_train_modelo_valid <- setdiff( dados_nerd_train_modelo, dados_nerd_train_modelo_train )

- Ajustes de poda para vários modelos de árvore em sequência

Usamos a amostra de treino para ajustar o modelo e a amostra de validação para selecionar os melhores e realizar o teste final na amostra de teste

In [ ]:
iteracoes <- 150 # numero de iteracoes para tunning
s_seeds <- sample(1000000:9999999, iteracoes) # sementes aleatorias
dados_amostra_avaliacao_questoes <- NULL
dados_amostra_avaliacao_pca <- NULL

In [ ]:
for ( iter in 1:iteracoes ){
  
  set.seed( s_seeds[iter] )
  minsplit_ <- sample(5:30, 1)
  cp_ <- runif(1, 0.001, 0.1)
  maxcompete_ <- sample(2:30, 1)
  maxdepth_ <- sample(10:50, 1)
  
  modelo_arvore_questoes <- rpart( formula = nerdy_classification ~ .,
                                   method = "class",
                                   data = dados_nerd_train_modelo_train %>% 
                                     dplyr::select( nerdy_classification, contains('Q') ),
                                   control = list( minsplit = minsplit_,
                                                   cp = cp_,
                                                   maxcompete = maxcompete_,
                                                   maxdepth = maxdepth_,
                                                   xval = 0 ) )
  
  modelo_arvore_pca <- rpart( formula = nerdy_classification ~ .,
                              method = "class",
                              data = dados_nerd_train_modelo_train %>% 
                                dplyr::select( nerdy_classification, contains('PC') ),
                              control = list( minsplit = minsplit_,
                                              cp = cp_,
                                              maxcompete = maxcompete_,
                                              maxdepth = maxdepth_,
                                              xval = 0 ) )
  
  pred_tree_questoes <- predict( modelo_arvore_questoes, dados_nerd_train_modelo_valid, type = 'class' )
  pred_tree_pca <- predict( modelo_arvore_pca, dados_nerd_train_modelo_valid, type = 'class' )
  
  
  acc_questoes <- confusionMatrix( pred_tree_questoes,
                   dados_nerd_train_modelo_valid$nerdy_classification,
                   positive = '1' )$overall[1]
  
  acc_pca <- confusionMatrix( pred_tree_pca,
                   dados_nerd_train_modelo_valid$nerdy_classification,
                   positive = '1' )$overall[1]

  
  aval_questoes <- data_frame( seed = s_seeds[iter],
                               minsplit_ = minsplit_,
                               cp_ = cp_,
                               maxcompete_ = maxcompete_,
                               maxdepth_ = maxdepth_,
                               acuracia = acc_questoes )
  
  aval_pca <- data_frame( seed = s_seeds[iter],
                          minsplit_ = minsplit_,
                          cp_ = cp_,
                          maxcompete_ = maxcompete_,
                          maxdepth_ = maxdepth_,
                          acuracia = acc_pca )
  
  dados_amostra_avaliacao_questoes <- bind_rows( dados_amostra_avaliacao_questoes, aval_questoes )
  dados_amostra_avaliacao_pca <- bind_rows( dados_amostra_avaliacao_pca, aval_pca )
  
}

- Salvamos os melhores modelos

In [ ]:
bests_questoes <- dados_amostra_avaliacao_questoes %>% 
  arrange( acuracia ) %>% 
  head(1)

bests_pca <- dados_amostra_avaliacao_pca %>% 
  arrange( acuracia ) %>% 
  head(1)

In [ ]:
bests_questoes
bests_pca

- Retreina os melhores modelos e guarda

In [ ]:
modelo_arvore_1 <- rpart( formula = nerdy_classification ~ ., 
                          data = dados_nerd_train_modelo_train %>% 
                            dplyr::select( nerdy_classification, contains('Q') ),
                          control = list( minsplit = bests_questoes$minsplit_[1],
                                          cp = bests_questoes$cp_[1],
                                          maxcompete = bests_questoes$maxcompete_[1],
                                          maxdepth = bests_questoes$maxdepth_[1],
                                          xval = 0 ) )

In [ ]:
modelo_arvore_2 <- rpart( formula = nerdy_classification ~ ., 
                          data = dados_nerd_train_modelo_train %>% 
                            dplyr::select( nerdy_classification, contains('PC') ),
                          control = list( minsplit = bests_pca$minsplit_[1],
                                          cp = bests_pca$cp_[1],
                                          maxcompete = bests_pca$maxcompete_[1],
                                          maxdepth = bests_pca$maxdepth_[1],
                                          xval = 0 ) )

## Avaliação dos modelos

- Criamos os PC`s usando os dados da amostra de teste

In [ ]:
scores_teste <- predict( pca_model, 
         dados_nerd_test %>% 
           dplyr::select( contains('Q') ) ) %>% 
  tbl_df()

- Adicionamos os PC`s na amostra de teste

In [ ]:
dados_nerd_test_modelo <- dados_nerd_test %>% 
  mutate( gender = as.factor(gender),
          education = as.factor(education),
          married = as.factor(married),
          ASD = as.factor(ASD),
          nerdy_classification = as.factor(nerdy_classification) ) %>% 
  bind_cols(., scores_teste[, 1:11] )

In [ ]:
dados_nerd_test_modelo %>% 
    head()

- Geramos as previsoes para avaliação

In [1]:
dados_avaliacao <- dados_nerd_test_modelo %>% 
  mutate( pred_nerdy_arvore_questoes = predict( modelo_arvore_1, ., type = 'class' ),
          pred_nerdy_arvore_pca = predict( modelo_arvore_2, ., type = 'class' ) ) %>% 
  dplyr::select( nerdy_classification, pred_nerdy_arvore_questoes, pred_nerdy_arvore_pca)

ERROR: Error in dados_nerd_test_modelo %>% mutate(pred_nerdy_arvore_questoes = as.factor(predict(modelo_arvore_1, : could not find function "%>%"


In [ ]:
dados_avaliacao %>% 
    head()

- Comparamos os modelos pela matriz de confusão e acurácia

Modelo 1: somente as 26 questões

In [ ]:
confusionMatrix( dados_avaliacao$pred_nerdy_arvore_questoes,
                 dados_avaliacao$nerdy_classification,
                 positive = '1' )

Modelo 2: somente os PC`s

In [ ]:
confusionMatrix( dados_avaliacao$pred_nerdy_arvore_pca,
                 dados_avaliacao$nerdy_classification,
                 positive = '1' )